In [0]:
!pip install scikit-optimize

     |████████████████████████████████| 81kB 7.0MB/s 


In [0]:
import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/gdrive')

import os
os.chdir("/gdrive/My Drive/Colab Notebooks/dacon")

In [0]:
resampled = pd.read_csv('./data/resampled_0825.csv',parse_dates=['date'])
resampled.drop('holiday',axis=1,inplace=True)
resampled.shape

In [0]:
naive_store_info = pd.read_csv('data/comp_9th/naive_store_info.csv')
naive_store_info = naive_store_info[['store_id','type_bus','high_region']]
print(resampled.shape)
resampled = resampled.merge(naive_store_info, on=['store_id'])
print(resampled.shape)

(1841687, 119)
(1841687, 121)


In [0]:
# resampled['cycle_idx_week1'] = (resampled['store_id'].astype(int).astype(str)) + (resampled['cycle_idx_week1'].astype(int).astype(str))
# resampled['cycle_idx_week4'] = (resampled['store_id'].astype(int).astype(str)) + (resampled['cycle_idx_week4'].astype(int).astype(str))
# resampled['cycle_idx_week8'] = (resampled['store_id'].astype(int).astype(str)) + (resampled['cycle_idx_week8'].astype(int).astype(str))


In [0]:
# resampled.drop(['cycle_idx_week1', 'cycle_idx_week4','cycle_idx_week8'], axis = 1 , inplace = True)

In [0]:
target = 'y'
cat_vars = ['store_id',  'Month', 'Week', 'Day', 'Dayofweek','Dayofyear',
            'weekofyear', 'type_bus', 'high_region','hday_name','is_holiday',
            'is_weekend','cur_zero','cycle_idx_week1', 'cycle_idx_week4', 'cycle_idx_week8']

cont_vars = [cols for cols in resampled.columns.values if cols not in cat_vars+[target,'date']]

ratios = [i for i in cont_vars if "ratio" in i]
non_ratios = [i for i in cont_vars if i not in ratios]


In [0]:
resampled[ratios] = resampled[ratios].replace(np.inf, 1e+17)
resampled[ratios] = resampled[ratios].replace(-np.inf, -1e+17)
resampled[non_ratios] = resampled[non_ratios].fillna(value=-1)
resampled[ratios] = resampled[ratios].fillna(value=0)
print(resampled[cont_vars].columns[resampled[cont_vars].isna().any()].tolist())

resampled = resampled.set_index('date')
resampled[cat_vars] = resampled[cat_vars].fillna(value='NaN')



In [0]:
print(resampled.shape)

resampled['type_bus'] = resampled['type_bus'].astype(str)
resampled['high_region'] = resampled['high_region'].astype(str)

%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler

encoders = {}
for v in cat_vars:
    le = LabelEncoder()
    le.fit(resampled[v].values)
    encoders[v] = le
    resampled.loc[:, v] = le.transform(resampled[v].values)
    #print('{0}: {1}'.format(v, le.classes_))

(1841687, 120)


In [0]:
d = [i.date() for i in pd.date_range('2017-01-01', '2018-10-31')]
import random
random.seed(2019)
idx = random.sample(d, 5)

train = resampled[(resampled.index <= pd.to_datetime('2018-11-22')) & 
                    (~resampled.index.isin(idx))]
val  = resampled[((resampled.index <= pd.to_datetime('2018-11-29')) &
                 (resampled.index  >= pd.to_datetime('2018-11-23'))) |
                 (resampled.index.isin(idx))]
real_test = resampled[resampled.index  == pd.to_datetime('2019-02-28')]

print(train.shape)
train = train[train.y > 0]
train = train[train.amount > 0]
print(train.shape)
print(val.shape)
val = val[val.y > 0]
print(val.shape)


(1639086, 120)
(1144174, 120)
(23604, 120)
(23448, 120)


In [0]:

X = train[cat_vars + cont_vars].copy()
X_val = val[cat_vars + cont_vars].copy()
y = train[target].copy()
y_val = val[target].copy()

X_test = real_test[cat_vars + cont_vars].copy()

scaler = StandardScaler()
X.loc[:, cont_vars] = scaler.fit_transform(X[cont_vars].values)
X_val.loc[:, cont_vars] = scaler.transform(X_val[cont_vars].values)
X_test.loc[:, cont_vars] = scaler.transform(X_test[cont_vars].values)


In [0]:
import gc
gc.collect()


50

In [0]:
for v in cat_vars:
    X[v] = X[v].astype('int').astype('category').cat.as_ordered()
    X_val[v] = X_val[v].astype('int').astype('category').cat.as_ordered()
    X_test[v] = X_test[v].astype('int').astype('category').cat.as_ordered()

for v in cont_vars:
    X[v] = X[v].astype('float32')
    X_val[v] = X_val[v].astype('float32')
    X_test[v] = X_test[v].astype('float32')    

In [0]:
cat_sizes = [(c, len(X[c].cat.categories)) for c in cat_vars]
cat_sizes

embedding_sizes = [(c, min(64, (c + 1) // 2)) for _, c in cat_sizes ]
#embedding_sizes = [(c, min(50, (c + 1) // 2)) for _, c in cat_sizes ]
embedding_sizes


[(1967, 64),
 (12, 6),
 (52, 26),
 (31, 16),
 (7, 4),
 (366, 64),
 (52, 26),
 (40, 20),
 (18, 9),
 (20, 10),
 (2, 1),
 (2, 1),
 (1, 1),
 (47, 24),
 (14, 7),
 (6, 3)]

In [0]:
X_array = []
X_val_array = []
X_test_array = []

for i, v in enumerate(cat_vars):
    X_array.append(X.iloc[:, i])
    X_val_array.append(X_val.iloc[:, i])
    X_test_array.append(X_test.iloc[:, i])

X_array.append(X.iloc[:, len(cat_vars):])
X_val_array.append(X_val.iloc[:, len(cat_vars):])
X_test_array.append(X_test.iloc[:, len(cat_vars):])

len(X_array), len(X_val_array), len(X_test_array)

(17, 17, 17)

In [0]:
import gc
gc.collect()

273

## 1. Entity Embedding



In [0]:
import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer

from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.models import Model
from keras.layers import Activation, BatchNormalization, Concatenate
from keras.layers import Dropout, Dense, Input, Reshape
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2, l1
import tensorflow


In [0]:
K.clear_session()
tensorflow.reset_default_graph()
gc.collect()

107

In [0]:
def EmbeddingNet(cat_vars, cont_vars, embedding_sizes, learning_rate, num_dense_layers,
                 num_dense_nodes, dr_rate):
    inputs = []
    embed_layers = []
    for (c, (in_size, out_size)) in zip(cat_vars, embedding_sizes):
        i = Input(shape=(1,))
        o = Embedding(in_size, out_size, name=c)(i)
        o = Reshape(target_shape=(out_size,))(o)
        inputs.append(i)
        embed_layers.append(o)


    embed = Concatenate()(embed_layers)
    embed = Dropout(dr_rate)(embed)

    cont_input = Input(shape=(len(cont_vars),))
    inputs.append(cont_input)

    x = Concatenate()([embed, cont_input])

    for i in range(num_dense_layers):
        name = 'layer_dense_{0}'.format(i+1)
        x = Dense(int(num_dense_nodes / (2**(i))) ,
                  kernel_initializer='he_normal')(x)
        x = Activation('relu')(x)
        x = Dropout(dr_rate)(x)

    x = Dense(1, kernel_initializer='he_normal')(x)
    x = Activation('linear')(x)

    model = Model(inputs=inputs, outputs=x)
    opt = Adam(lr=learning_rate)
    model.compile(loss='mean_absolute_error', optimizer=opt)

    return model

In [0]:
from CLR.clr_callback import CyclicLR
import tensorflow
import time

dim_learning_rate = Real(low=1e-5, high=1e-3, prior='log-uniform', 
                         name='learning_rate')                           # 0.00005
dim_num_dense_layers = Integer(low=2, high=4, name='num_dense_layers')   # 3
dim_num_dense_nodes = Integer(low=500, high=1024, name='num_dense_nodes')# 512
dim_dr_rate = Real(low=0.05, high=0.5,name="dr_rate")                    # 0.2

dimensions = [dim_learning_rate,
              dim_num_dense_layers, 
              dim_num_dense_nodes,
              dim_dr_rate
             ]

## 2. Cyclic Learning Rate


In [0]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers,num_dense_nodes, dr_rate):
    print("params")
    print(learning_rate, num_dense_layers, num_dense_nodes, dr_rate)
    print()
    clr = CyclicLR(base_lr=learning_rate, max_lr=learning_rate*6,
                        step_size=1024., mode='exp_range',
                        gamma=0.99994)

    model = EmbeddingNet(cat_vars, cont_vars, embedding_sizes,
                         learning_rate=learning_rate,
                         num_dense_layers=num_dense_layers,
                         num_dense_nodes=num_dense_nodes,
                         dr_rate = dr_rate
                        )
    
    start_time = time.time()
    
    EPOCHS = 20
    #named blackbox becuase it represents the structure
    blackbox = model.fit(x=X_array,
                        y=y,
                        epochs=EPOCHS,
                        batch_size=512,
                        callbacks=[clr,],
                        validation_data=(X_val_array, y_val),
                        shuffle = True
                        )
    
    val_mae = min(blackbox.history['val_loss'])
    print("--- %s seconds ---" % (time.time() - start_time))
    # Print the classification accuracy.
    print("val_loss: {0}".format(val_mae))
    print()
    del model

    K.clear_session()
    tensorflow.reset_default_graph()
    gc.collect()
    return val_mae

In [0]:
K.clear_session()
tensorflow.reset_default_graph()
gc.collect()



107

## 3. Bayesian Optimization

In [0]:
K.clear_session()
tensorflow.reset_default_graph()
default_parameters = [0.00005, 3, 512, 0.2]
res_gp = gp_minimize(func=fitness,
                     dimensions=dimensions,
                     n_calls = 15,
                     n_jobs=-1,
                     kappa = 5,
                     x0=default_parameters)

report = pd.concat([pd.DataFrame(
                  res_gp.x_iters, 
                  columns = ["learning_rate", "num_dense_layers","num_dense_nodes","dr_rate"]),
                  (pd.Series(res_gp.func_vals, name="mae"))], axis=1)

report.to_csv('./submit_0825/report_nn_skopt_0828_no0.csv', index = False)


params
5e-05 3 512 0.2

Train on 1144174 samples, validate on 23448 samples
Epoch 1/20
1144174/1144174 [==============================] - 37s 32us/step - loss: 1790079.2643 - val_loss: 790977.9941
Epoch 2/20
1144174/1144174 [==============================] - 36s 31us/step - loss: 802364.9909 - val_loss: 739542.5065
Epoch 3/20
1144174/1144174 [==============================] - 36s 31us/step - loss: 760595.5979 - val_loss: 729153.9583
Epoch 4/20
1144174/1144174 [==============================] - 36s 31us/step - loss: 742083.4323 - val_loss: 726254.8358
Epoch 5/20
1144174/1144174 [==============================] - 36s 31us/step - loss: 731187.7068 - val_loss: 724048.2447
Epoch 6/20
1144174/1144174 [==============================] - 36s 31us/step - loss: 723739.8087 - val_loss: 725567.3215
Epoch 7/20
1144174/1144174 [==============================] - 36s 31us/step - loss: 720053.2920 - val_loss: 725089.9728
Epoch 8/20
1144174/1144174 [==============================] - 36s 31us/step - loss:

In [0]:
report.sort_values('mae')

,learning_rate,num_dense_layers,num_dense_nodes,dr_rate,mae
5,0.000934,3,934,0.358871,468886.612494
3,0.000768,4,823,0.438239,480870.252431
12,0.000877,4,948,0.415393,482306.894138
14,0.000879,4,818,0.426267,490173.560986
13,0.000888,3,899,0.415041,490476.096192
11,0.000376,4,643,0.453949,585987.701073
9,0.000305,3,720,0.175679,635160.005992
6,0.000106,4,564,0.160238,670485.670963
4,0.000339,2,913,0.300672,704834.939888
1,0.000055,4,749,0.403756,705453.598942


## 4. Train & Test Example

In [0]:

from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.models import Model
from keras.layers import Activation, BatchNormalization, Concatenate
from keras.layers import Dropout, Dense, Input, Reshape
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2, l1
import tensorflow

import gc

In [0]:
from CLR.clr_callback import CyclicLR
import tensorflow
import time

In [0]:
def EmbeddingNet(cat_vars, cont_vars, embedding_sizes, learning_rate, num_dense_layers,
                 num_dense_nodes, dr_rate):
    inputs = []
    embed_layers = []
    for (c, (in_size, out_size)) in zip(cat_vars, embedding_sizes):
        i = Input(shape=(1,))
        o = Embedding(in_size, out_size, name=c)(i)
        o = Reshape(target_shape=(out_size,))(o)
        inputs.append(i)
        embed_layers.append(o)


    embed = Concatenate()(embed_layers)
    embed = Dropout(dr_rate)(embed)

    cont_input = Input(shape=(len(cont_vars),))
    inputs.append(cont_input)

    x = Concatenate()([embed, cont_input])

    for i in range(num_dense_layers):
        name = 'layer_dense_{0}'.format(i+1)
        x = Dense(int(num_dense_nodes / (2**(i))) ,
                  kernel_initializer='he_normal')(x)
        x = Activation('relu')(x)
        x = Dropout(dr_rate)(x)

    x = Dense(1, kernel_initializer='he_normal')(x)
    x = Activation('linear')(x)

    model = Model(inputs=inputs, outputs=x)
    opt = Adam(lr=learning_rate)
    model.compile(loss='mean_absolute_error', optimizer=opt)

    return model

In [0]:
LR = 0.0003 
NDL = 2
NDN =  800
DR = 0.35

model_weight_loc = './BM_weights_0828_check.hdf5'


model = EmbeddingNet(cat_vars, cont_vars, embedding_sizes,
                     learning_rate=LR,
                     num_dense_layers=NDL,
                     num_dense_nodes=NDN,
                     dr_rate = DR
                    )


In [0]:
from CLR.clr_callback import CyclicLR

clr = CyclicLR(base_lr=LR, max_lr=LR*6,
                    step_size=1024., mode='exp_range',
                    gamma=0.99994)


checkpoint = ModelCheckpoint(model_weight_loc, monitor='val_loss',
                             save_best_only=True)

EPOCHS = 40
#named blackbox becuase it represents the structure
blackbox = model.fit(x=X_array,
                    y=y,
                    epochs=EPOCHS,
                    batch_size=512,
                    callbacks=[clr, checkpoint],
                    validation_data=(X_val_array, y_val),
                    shuffle = True
                    )

val_mae = min(blackbox.history['val_loss'])

In [0]:
from sklearn.metrics import mean_absolute_error
from keras.models import load_model

saved_model = EmbeddingNet(cat_vars, cont_vars, embedding_sizes,
                     learning_rate=LR,
                     num_dense_layers=NDL,
                     num_dense_nodes=NDN,
                     dr_rate = DR
                    )

saved_model.load_weights(model_weight_loc)

pred =  saved_model.predict(x = X_test_array)

store_id_inversed = encoders['store_id'].inverse_transform(X_test.store_id.values)

predicted = pd.DataFrame(
    {'store_id': store_id_inversed,
     'pred': pred.reshape(len(pred))
    }
)


- References:
    - Entity Embedding
        - https://arxiv.org/pdf/1604.06737.pdf
        - https://www.johnwittenauer.net/deep-learning-with-keras-structured-time-series/
        - https://github.com/fastai/fastai/blob/master/courses/dl1/lesson3-rossman.ipynb
    
    - Cyclic Learning Rate
        - https://arxiv.org/abs/1506.01186
        - https://github.com/bckenstler/CLR

    - Bayesian Optimization
        - https://scikit-optimize.github.io/#skopt.gp_minimize
        - https://medium.com/@crawftv/parameter-hyperparameter-tuning-with-bayesian-optimization-7acf42d348e1



